In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType

spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",IntegerType(),True), \
        StructField("FINAL_CAR_KEY", IntegerType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

22/02/12 22:51:20 WARN Utils: Your hostname, amin-X556UQK resolves to a loopback address: 127.0.1.1; using 192.168.1.100 instead (on interface wlp3s0)
22/02/12 22:51:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/12 22:51:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/12 22:51:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.csv('Traffic.csv',header=True,schema=schema)
df.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
|        155|       81|       87625017|     87625017|               5|       161|2021-06-01 04:14:21|
|     631757|       81|        8652928|      8652928|               5|       161|2021-06-01 03:58:57|
|     631757|       81|        8548123|      8548123|               5|       161|2021-06-01 04:01:38|
|     631757|       81|       24715264|     24715264|               5|       161|2021-06-01 03:56:57|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 5 rows



In [5]:
schema_company = StructType([ \
        StructField("COMPANY_ID", IntegerType(), True), 
        StructField("COMPANY_NAME", StringType(), True), \
    ])
        # StructField("ActiveSystemID", TimestampType(), True)
companies = spark.read.csv('CompanyID.csv',header=True,schema=schema_company)
companies.show(5)

+----------+---------------+
|COMPANY_ID|   COMPANY_NAME|
+----------+---------------+
|        61|دید افزار سیستم|
|        62|           فناپ|
|       141|     شرکت‌ همسو|
|       161|        دنا‌ وب|
|       181|   هوشمند‌ آسیا|
+----------+---------------+
only showing top 5 rows



22/02/12 22:59:51 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/amin/amin/University/Term7/MDA/HW/project/CompanyID.csv


In [3]:
DATA_COUNT = df.count()
CAR_COUNT = df.select('FINAL_CAR_KEY').distinct().count()
CAMERA_COUNT = df.select('DEVICE_CODE').distinct().count()
print('all count: ', DATA_COUNT)
print('car count: ', CAR_COUNT)
print('camera count: ', CAMERA_COUNT)

all count:  34989160
car count:  5487645
camera count:  1035


In [4]:
car_counts = df.groupBy('FINAL_CAR_KEY').count()
car_counts.sort('count', ascending=False).show(10)

22/02/11 22:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:47:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:47:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:47:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------+-------+
|FINAL_CAR_KEY|  count|
+-------------+-------+
|     64111706|1891912|
|     69177480|  80818|
|      8073331|  30194|
|     67682391|   6227|
|      7633319|   1579|
|     14919817|   1513|
|     19516092|   1385|
|      8396536|    804|
|     73138295|    730|
|     23975824|    550|
+-------------+-------+
only showing top 10 rows



In [3]:
car_camera_counts = df.groupBy('FINAL_CAR_KEY', 'DEVICE_CODE').count()
car_camera_counts.sort('count', ascending=False).show(20)

+-------------+-----------+------+
|FINAL_CAR_KEY|DEVICE_CODE| count|
+-------------+-----------+------+
|     64111706|   22010039|101540|
|     64111706|   22010048| 74714|
|     64111706|   22010061| 73444|
|     64111706|   22010040| 51751|
|     64111706|   22010078| 49628|
|     64111706|   22010047| 43255|
|     64111706|   22010059| 39823|
|     64111706|   22010053| 39176|
|     64111706|   22010054| 38625|
|     64111706|   22010083| 34779|
|     64111706|   22010060| 33991|
|     64111706|   22010058| 31256|
|     64111706|     631640| 30567|
|     64111706|   22010079| 30521|
|     64111706|   22010080| 30356|
|     64111706|   22010088| 24077|
|     64111706|  100701119| 21319|
|     69177480|   22010061| 19105|
|     64111706|   22010094| 16564|
|     64111706|   22010087| 15686|
+-------------+-----------+------+
only showing top 20 rows



In [22]:
df.summary().show()

+-------+--------------------+------------------+--------------------+-------------------+------------------+------------------+
|summary|         DEVICE_CODE|         SYSTEM_ID|     ORIGINE_CAR_KEY|      FINAL_CAR_KEY|  CHECK_STATUS_KEY|        COMPANY_ID|
+-------+--------------------+------------------+--------------------+-------------------+------------------+------------------+
|  count|            34989160|          34989160|            34989160|           34989159|          34989159|          34989160|
|   mean|2.2262293326161418E7|236.54757084765683|3.6067905310963854E7|3.702608833010948E7|5.2871780656402745|218.93599797765935|
| stddev| 3.741603259148462E7| 84.44216726147245|3.1140705482597664E7|3.076635128051584E7|0.6188797307503013| 63.36385080523604|
|    min|                 100|                81|           100000013|          100000013|                 1|                 0|
|    25%|              900107|               282|           9713210.0|          9839529.0|       

##### Most Crimes

In [3]:
from pyspark.sql.functions import hour,dayofweek, count
from pyspark.sql.window import Window

window = Window.partitionBy('FINAL_CAR_KEY')
df_with_car_count = df.withColumn('count', count('FINAL_CAR_KEY').over(window))
df_cleaned = df_with_car_count.filter(df_with_car_count['count'] < 1000)
df_cleaned.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+-----+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|count|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+-----+
|     803001|      102|        7631929|      7631929|               6|       161|2021-06-01 10:37:55|    1|
|     900158|      283|        7631979|      7631979|               5|       264|2021-06-06 12:30:50|   11|
|  100700881|      283|        7631979|      7631979|               5|       264|2021-06-06 17:46:08|   11|
|     900222|      283|        7631979|      7631979|               5|       264|2021-06-06 17:49:52|   11|
|  100700820|      283|        7631979|      7631979|               5|       264|2021-06-06 22:45:18|   11|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+-----+
only showing top 5 rows



In [8]:
crimes_df = df_cleaned.filter(~df_cleaned.SYSTEM_ID.isin([84,283,284]))
crimes_count_df = crimes_df.groupBy('FINAL_CAR_KEY').count()
crimes_count_df.orderBy('count', ascending=False).show(20)

22/02/11 22:55:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 22:55:42 WARN RowBasedKeyValueBatch: Calling spill() on

+-------------+-----+
|FINAL_CAR_KEY|count|
+-------------+-----+
|     73138295|  730|
|     23975824|  539|
|     28621897|  508|
|      9288458|  462|
|     67483228|  393|
|      8137760|  362|
|     43211832|  351|
|     10195991|  344|
|     21683458|  313|
|      9093773|  306|
|     10146615|  293|
|     17890990|  292|
|      7822296|  287|
|      8772489|  281|
|     20143061|  261|
|      7685553|  260|
|      8140950|  258|
|     35811639|  252|
|      8783398|  240|
|      7713151|  238|
+-------------+-----+
only showing top 20 rows



##### Best Cameras

In [14]:
from pyspark.sql.functions import when, col
df_with_corrects = df_cleaned.withColumn('correct', when(col('ORIGINE_CAR_KEY') == col('FINAL_CAR_KEY'), 1).otherwise(0))
df_with_corrects.show(100)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+-----+-------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|count|correct|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+-----+-------+
|     803001|      102|        7631929|      7631929|               6|       161|2021-06-01 10:37:55|    1|      1|
|     900158|      283|        7631979|      7631979|               5|       264|2021-06-06 12:30:50|   11|      1|
|  100700881|      283|        7631979|      7631979|               5|       264|2021-06-06 17:46:08|   11|      1|
|     900222|      283|        7631979|      7631979|               5|       264|2021-06-06 17:49:52|   11|      1|
|  100700820|      283|        7631979|      7631979|               5|       264|2021-06-06 22:45:18|   11|      1|
|     900194|      283|        7631979|      7631979|               6|  

In [16]:
success_rate_df = df_with_corrects.groupBy('DEVICE_CODE').agg({
    '*': 'count',
    'correct': 'sum'
})
success_rate_df.show(5)

22/02/11 23:18:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 23:18:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------+------------+--------+
|DEVICE_CODE|sum(correct)|count(1)|
+-----------+------------+--------+
|     631926|         562|     685|
|        148|        3631|    3631|
|     631488|         592|     592|
|     811803|         476|     476|
|    1001043|         425|     483|
+-----------+------------+--------+
only showing top 5 rows



In [19]:
success_rate_df = success_rate_df.withColumn('correct_rate', success_rate_df['sum(correct)'] / success_rate_df['count(1)'])
success_rate_df.show(5)

22/02/11 23:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 23:20:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------+------------+--------+------------------+
|DEVICE_CODE|sum(correct)|count(1)|      correct_rate|
+-----------+------------+--------+------------------+
|     631926|         562|     685|0.8204379562043795|
|        148|        3631|    3631|               1.0|
|     631488|         592|     592|               1.0|
|     811803|         476|     476|               1.0|
|    1001043|         425|     483|0.8799171842650103|
+-----------+------------+--------+------------------+
only showing top 5 rows



In [20]:
success_rate_df_sorted = success_rate_df.sort('correct_rate', ascending=False)
success_rate_df_sorted.show(5)

22/02/11 23:22:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 23:22:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------+------------+--------+------------+
|DEVICE_CODE|sum(correct)|count(1)|correct_rate|
+-----------+------------+--------+------------+
|     900232|       56668|   56668|         1.0|
|     200901|        3628|    3628|         1.0|
|     900203|      136885|  136885|         1.0|
|        148|        3631|    3631|         1.0|
|     631488|         592|     592|         1.0|
+-----------+------------+--------+------------+
only showing top 5 rows



In [21]:
success_rate_df_sorted.select('correct_rate').summary().show()

22/02/11 23:23:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/02/11 23:23:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------+-------------------+
|summary|       correct_rate|
+-------+-------------------+
|  count|               1033|
|   mean| 0.9436302470613623|
| stddev|0.09830451187865522|
|    min|               0.46|
|    25%| 0.8947368421052632|
|    50%|                1.0|
|    75%|                1.0|
|    max|                1.0|
+-------+-------------------+



##### Best Companies

In [6]:

from pyspark.sql.functions import when, col
df_with_corrects = df_cleaned.withColumn('correct', when(col('ORIGINE_CAR_KEY') == col('FINAL_CAR_KEY'), 1).otherwise(0))

# df_with_corrects = df_with_corrects.join(companies, df_with_corrects.COMPANY_ID == companies.COMPANY_ID)

success_rate_df = df_with_corrects.groupBy('COMPANY_ID').agg({
    '*': 'count',
    'correct': 'sum'
})
success_rate_df = success_rate_df.withColumn('correct_rate', success_rate_df['sum(correct)'] / success_rate_df['count(1)'])
success_rate_df = success_rate_df.join(companies, success_rate_df.COMPANY_ID == companies.COMPANY_ID)
success_rate_df_sorted = success_rate_df.sort('correct_rate', ascending=False).take(5)


22/02/12 23:03:12 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/amin/amin/University/Term7/MDA/HW/project/CompanyID.csv
22/02/12 23:03:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [7]:
success_rate_df_sorted

[Row(COMPANY_ID='264', sum(correct)=20837743, count(1)=20837743, correct_rate=1.0, COMPANY_ID=264, COMPANY_NAME='بصیر\u200c پردازش'),
 Row(COMPANY_ID='161', sum(correct)=10943676, count(1)=10943936, correct_rate=0.999976242551126, COMPANY_ID=161, COMPANY_NAME='دنا\u200c وب'),
 Row(COMPANY_ID='61', sum(correct)=13337, count(1)=14038, correct_rate=0.9500641116968229, COMPANY_ID=61, COMPANY_NAME='دید افزار سیستم'),
 Row(COMPANY_ID='274', sum(correct)=126779, count(1)=141735, correct_rate=0.8944791335943839, COMPANY_ID=274, COMPANY_NAME='داتیس'),
 Row(COMPANY_ID='181', sum(correct)=15244, count(1)=18764, correct_rate=0.81240673630356, COMPANY_ID=181, COMPANY_NAME='هوشمند\u200c آسیا')]